In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

PATH = "data/dogbreeds/"
sz=224
arch=resnext50
bs=64


In [9]:
label_csv=f'{PATH}labels.csv'
n=len(list(open(label_csv)))-1
val_idxs=get_cv_idxs(n)

In [18]:
arch=resnet34
data = ImageClassifierData.from_csv(PATH,'train',f'{PATH}labels.csv' ,test_name='test',tfms=tfms_from_model(arch, sz),val_idxs=val_idxs, suffix='.jpg',bs=bs)
learn = ConvLearner.pretrained(arch, data, precompute=True,ps=0.5)
learn.fit(0.01, 7)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

[0.      2.67204 1.32785 0.74525]                           
[1.      1.41368 0.78824 0.79808]                           
[2.      1.02726 0.6448  0.82695]                           
[3.      0.84263 0.58277 0.82695]                            
[4.      0.77939 0.54961 0.83086]                            
[5.      0.67363 0.53241 0.83135]                            
[6.      0.63006 0.52017 0.83574]                            



In [19]:
learn.precompute=False
learn.fit(1e-2, 3, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

[0.      0.59861 0.5057  0.83818]                            
[1.      0.58526 0.50552 0.8318 ]                            
[2.      0.57313 0.49605 0.83864]                            



In [ ]:
learn.unfreeze()
lr=np.array([1e-4,1e-3,1e-2])
learn.fit(lr, 3, cycle_len=1)

In [13]:
log_preds,y = learn.TTA(is_test=True)
probs = np.mean(np.exp(log_preds), axis=0)

In [12]:
accuracy(probs,y), metrics.log_loss(y, probs)

(0.8136007827788649, 0.6079921600792129)

In [14]:
probs[:10]

array([[0.00002, 0.00001, 0.00002, ..., 0.00002, 0.00008, 0.00001],
       [0.00379, 0.01022, 0.00071, ..., 0.00023, 0.0017 , 0.00414],
       [0.00066, 0.00002, 0.00016, ..., 0.00004, 0.00003, 0.00046],
       ...,
       [0.     , 0.     , 0.     , ..., 0.     , 0.00001, 0.     ],
       [0.00018, 0.00021, 0.00037, ..., 0.00013, 0.00617, 0.00069],
       [0.00011, 0.00546, 0.00074, ..., 0.00233, 0.00025, 0.00005]], dtype=float32)

In [23]:
df = pd.DataFrame(probs)

df.columns = data.classes
df.insert(0,'id', [o[5:-4] for o in data.test_ds.fnames])

In [21]:
SUBM = f'{PATH}sub/'
os.makedirs(SUBM, exist_ok=True)

In [24]:
df.head

<bound method NDFrame.head of                                      id  affenpinscher  afghan_hound  \
0      57df9fc7793c4c40e806167ea94773a5   1.807428e-05      0.000008   
1      56883df95a5356a29297c6dccfab0b47   3.786643e-03      0.010223   
2      f12b509213db8bb0a721fcd228b31f0e   6.648732e-04      0.000018   
3      a59fa6f74528014c4ffe94e117868abe   8.324067e-06      0.000017   
4      d6da712fa79d6631324452fa74d648a3   2.534557e-05      0.000069   
5      6a2cfab843ba8a78ab3f179a56b9d8e2   4.456642e-04      0.041087   
6      a057d4b2a4343cb814937b517fb3be9a   6.078827e-07      0.000106   
7      957379d51e207de4885b41d0b1758d8c   9.925973e-07      0.000001   
8      56769dd3037ce85d52c2ef9b951137de   1.801961e-04      0.000210   
9      f90f630e3728dfe9428e291af0cc4af5   1.050842e-04      0.005464   
10     86c2d71776b76d0c7acdf34a87b7f9c7   1.508106e-05      0.000019   
11     0cf7c74a10aa9c602d194717ab781794   7.345284e-04      0.000782   
12     cf64016d08aacd4e9b6af996aa2

In [26]:
df.to_csv(f'{SUBM}subm.gz', compression='gzip', index=False)

In [27]:
FileLink(f'{SUBM}subm.gz')


/home/vaibhavnandwani/fastai/courses/dl1/data/dogbreeds/sub/subm.gz